In [6]:
AGENT_NAME = 'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500'
OBS_NAME = 'bb results/bb obs'

from stable_baselines3 import PPO

import pandas as pd
import numpy as np

import KBMproject.utilities as utils

import torch
import torch.nn.functional as F

In [7]:
agent = PPO.load(path=f"{AGENT_NAME}")

#actor = utils.extract_actor(agent)

In [8]:
df_adv_obs = pd.read_csv(f'{OBS_NAME}.csv', 
                         index_col=0,
                         header=0,
                         dtype='float32')
df_adv_obs.set_index(df_adv_obs.index.astype(int), inplace=True)

In [9]:
from art.estimators.classification import PyTorchClassifier as classifier
from torch.nn import CrossEntropyLoss
policy = classifier(
                    model=utils.extract_actor(agent),
                    loss=CrossEntropyLoss(), 
                    nb_classes=agent.action_space[0].n,
                    input_shape=agent.observation_space.shape,
                    device_type='gpu',
                    clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max())
                    )

In [10]:
est_logits = policy.predict(df_adv_obs.values, training_mode=False)
est_actions = np.argmax(est_logits, axis=1)
est_logits.shape

(8759, 20)

In [12]:
#np.savetxt(f'{OBS_NAME}_logits.csv',est_logits, delimiter=',')
np.savetxt(f'{OBS_NAME}_a.csv',est_actions, delimiter=',')

shows that using an art estimator gives the same results as SB3

In [ ]:
# agent_actions = df_adv_obs.apply(lambda row: agent.predict(row, deterministic=True)[0], axis='columns')
# agent_actions = np.concatenate(agent_actions.to_numpy())
# sum(est_actions!=agent_actions)

0

In [10]:
confidence = F.softmax(torch.from_numpy(est_logits), dim=-1)

In [11]:
df_a_conf = pd.DataFrame({'confidence':confidence.max(dim=1).values.numpy(), 'action':confidence.max(dim=1).indices.numpy()})

In [12]:
df_a_conf.groupby('action').describe()

confidence                                                              \
            count      mean       std       min       25%       50%       75%   
action                                                                          
0            30.0  0.616390  0.164534  0.266622  0.521663  0.597109  0.692668   
1           113.0  0.708356  0.190495  0.344390  0.551912  0.699424  0.899482   
2            94.0  0.702286  0.205128  0.280540  0.526543  0.715779  0.890393   
3           326.0  0.701107  0.188764  0.239396  0.545528  0.691467  0.880517   
4            26.0  0.687212  0.190755  0.345161  0.575195  0.739523  0.843946   
5            44.0  0.627038  0.178561  0.294040  0.477325  0.623936  0.777822   
6           389.0  0.677726  0.182171  0.247008  0.526139  0.667169  0.832041   
7           717.0  0.795438  0.190584  0.289945  0.650328  0.848983  0.971762   
8          2247.0  0.859724  0.167260  0.273913  0.758099  0.942228  0.994039   
9          1744.0  0.844827  0.174160  0.258846  0.730706  0.923078  0.990153   
10          124.0  0.657731  0.196124  0.263745  0.497357  0.636875  0.837897   
11          467.0  0.765224  0.196341  0.305732  0.589842  0.802010  0.948984   
12         1174.0  0.829808  0.174633  0.300799  0.717220  0.894473  0.979547   
13          592.0  0.812660  0.187521  0.282403  0.682664  0.876086  0.988038   
14           67.0  0.711893  0.202856  0.288499  0.526723  0.682724  0.922787   
15          222.0  0.699723  0.199184  0.269244  0.545069  0.715804  0.886241   
16          120.0  0.711407  0.195113  0.334382  0.530397  0.739647  0.884863   
17          159.0  0.676432  0.195470  0.275402  0.509938  0.664276  0.857850   
19          103.0  0.665335  0.189400  0.322099  0.508416  0.669255  0.831204   

                  
             max  
action            
0       0.935465  
1       0.999612  
2       0.998519  
3       0.998906  
4       0.965135  
5       0.984040  
6       0.993936  
7       0.999999  
8       1.000000  
9       1.000000  
10      0.992242  
11      0.999780  
12      0.999967  
13      1.000000  
14      0.999691  
15      0.999243  
16      0.998340  
17      0.999676  
19      0.983238

Note that some action are chosen with very low confidence

In [ ]:
df_a_conf.to_csv(f'{OBS_NAME} action confidence.csv')